In [79]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vaultdr
%env VAULT_HELM_RELEASE_NAME=vaultdr
%env VAULT_SERVICE_NAME=vaultdr-internal 
%env K8S_CLUSTER_NAME=cluster.local

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vaultdr
env: VAULT_HELM_RELEASE_NAME=vaultdr
env: VAULT_SERVICE_NAME=vaultdr-internal
env: K8S_CLUSTER_NAME=cluster.local


In [80]:
%%bash
export VAULT_K8S_NAMESPACE="vaultdr" \
export VAULT_HELM_RELEASE_NAME="vaultdr" \
export VAULT_SERVICE_NAME="vaultdr-internal" \
export K8S_CLUSTER_NAME="cluster.local" \
export WORKDIR=/tmp/vault


cat > ${WORKDIR}/vaultdr-csr.conf <<EOF
[req]
default_bits = 2048
prompt = no
encrypt_key = yes
default_md = sha256
distinguished_name = kubelet_serving
req_extensions = v3_req
[ kubelet_serving ]
O = system:nodes
CN = system:node:*.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth, clientAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = *.${VAULT_SERVICE_NAME}
DNS.2 = *.${VAULT_SERVICE_NAME}.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
DNS.3 = *.${VAULT_HELM_RELEASE_NAME}
DNS.4 = *.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
IP.1 = 127.0.0.1
EOF

openssl req -new -key ${WORKDIR}/vault.key -out ${WORKDIR}/vaultdr.csr -config ${WORKDIR}/vaultdr-csr.conf


cat > ${WORKDIR}/csrpr.yaml <<EOF
apiVersion: certificates.k8s.io/v1
kind: CertificateSigningRequest
metadata:
   name: vaultdr.svc
spec:
   signerName: kubernetes.io/kubelet-serving
   expirationSeconds: 8640000
   request: $(cat ${WORKDIR}/vaultdr.csr|base64|tr -d '\n')
   usages:
   - digital signature
   - key encipherment
   - server auth
EOF

kubectl create -f ${WORKDIR}/csrpr.yaml

certificatesigningrequest.certificates.k8s.io/vaultdr.svc created


In [81]:
%%bash
kubectl certificate approve vaultdr.svc

certificatesigningrequest.certificates.k8s.io/vaultdr.svc approved


In [82]:
%%bash
kubectl get csr vaultdr.svc

NAME          AGE   SIGNERNAME                      REQUESTOR       REQUESTEDDURATION   CONDITION
vaultdr.svc   0s    kubernetes.io/kubelet-serving   minikube-user   100d                Approved,Issued


In [83]:
%%bash
kubectl get csr vaultdr.svc -o jsonpath='{.status.certificate}' | openssl base64 -d -A -out ${WORKDIR}/vaultdr.crt

In [84]:
%%bash

kubectl create namespace $VAULT_K8S_NAMESPACE

kubectl create secret generic vault-ha-tls \
   -n $VAULT_K8S_NAMESPACE \
   --from-file=vault.key=${WORKDIR}/vault.key \
   --from-file=vault.crt=${WORKDIR}/vaultdr.crt \
   --from-file=vault.ca=${WORKDIR}/vault.ca


namespace/vaultdr created
secret/vault-ha-tls created


In [85]:
%%bash 
secret=$(cat /Users/jose/Demo/Vault_ADP/ADP_PlayGround/vault.hclic)
kubectl create secret generic vault-ent-license --from-literal="license=${secret}" -n $VAULT_K8S_NAMESPACE

secret/vault-ent-license created


In [86]:
%%bash
cat > ${WORKDIR}/overrides_pr.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding
injector:
   enabled: false
server:
# config.yaml
   image:
      repository: hashicorp/vault-enterprise
      tag: 1.14.1-ent
   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
      VAULT_CLIENT_CERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_CLIENT_KEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: ""
   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            cluster_name = "vault-dr"
            listener "tcp" {
               tls_disable = 0 # Disabling TLS to avoid issues when connecting to Vault via port forwarding
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vaultdr label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vaultdr-0.vaultdr-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            disable_mlock = true
            service_registration "kubernetes" {}
EOF

In [87]:
%%bash
helm install -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides_pr.yaml

NAME: vaultdr
LAST DEPLOYED: Thu Sep 14 07:42:15 2023
NAMESPACE: vaultdr
STATUS: deployed
REVISION: 1
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vaultdr. To learn more about the release, try:

  $ helm status vaultdr
  $ helm get manifest vaultdr


In [88]:
%%bash
kubectl get events -n vaultdr

LAST SEEN   TYPE      REASON                  OBJECT                                 MESSAGE
0s          Normal    ExternalProvisioning    persistentvolumeclaim/data-vaultdr-0   waiting for a volume to be created, either by external provisioner "k8s.io/minikube-hostpath" or manually created by system administrator
0s          Normal    Provisioning            persistentvolumeclaim/data-vaultdr-0   External provisioner is provisioning volume for claim "vaultdr/data-vaultdr-0"
0s          Normal    ProvisioningSucceeded   persistentvolumeclaim/data-vaultdr-0   Successfully provisioned volume pvc-02fd8e8d-d843-4ed5-9f01-639c5b4981af
0s          Normal    ExternalProvisioning    persistentvolumeclaim/data-vaultdr-1   waiting for a volume to be created, either by external provisioner "k8s.io/minikube-hostpath" or manually created by system administrator
0s          Normal    Provisioning            persistentvolumeclaim/data-vaultdr-1   External provisioner is provisioning volume for claim 

In [89]:
%%bash
sleep 10
kubectl exec -n $VAULT_K8S_NAMESPACE vaultdr-0 -- vault operator init \
    -key-shares=1 \
    -key-threshold=1 \
    -format=json > ${WORKDIR}/clusterdr-keys.json

In [90]:
%%bash
jq -r ".unseal_keys_b64[]" ${WORKDIR}/clusterdr-keys.json
VAULT_UNSEAL_KEY_PR=$(jq -r ".unseal_keys_b64[]" ${WORKDIR}/clusterdr-keys.json)

n+bu3nIJIK5djoNYG/LbA7tct2FLKb/F3IZY0/YNJYM=


In [91]:
%%bash
sleep 5
kubectl exec -n $VAULT_K8S_NAMESPACE vaultdr-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/clusterdr-keys.json)

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.14.1+ent
Build Date              2023-07-21T23:04:42Z
Storage Type            raft
Cluster Name            vault-dr
Cluster ID              1ffdecb3-8ea1-7553-68b8-530c69fff8d1
HA Enabled              true
HA Cluster              https://vaultdr-0.vaultdr-internal:8201
HA Mode                 active
Active Since            2023-09-14T05:42:39.839308926Z
Raft Committed Index    78
Raft Applied Index      78
Last WAL                32


In [93]:
%%bash
sleep 5
kubectl exec -n $VAULT_K8S_NAMESPACE vaultdr-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/clusterdr-keys.json)

Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.14.1+ent
Build Date         2023-07-21T23:04:42Z
Storage Type       raft
HA Enabled         true


In [94]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vaultdr-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/clusterdr-keys.json)

Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.14.1+ent
Build Date         2023-07-21T23:04:42Z
Storage Type       raft
HA Enabled         true


In [95]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vaultdr-0 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.14.1+ent
Build Date              2023-07-21T23:04:42Z
Storage Type            raft
Cluster Name            vault-dr
Cluster ID              1ffdecb3-8ea1-7553-68b8-530c69fff8d1
HA Enabled              true
HA Cluster              https://vaultdr-0.vaultdr-internal:8201
HA Mode                 active
Active Since            2023-09-14T05:42:39.839308926Z
Raft Committed Index    108
Raft Applied Index      108
Last WAL                35


In [96]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vaultdr-0 --  vault license inspect

Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

In [97]:
%%bash
cat ${WORKDIR}/clusterdr-keys.json | jq -r ".root_token"

hvs.7uWHdFJVOGSwjZrLP8O200QK


In [98]:
%%bash
#kubectl -n $VAULT_K8S_NAMESPACE get service vault
kubectl -n vaultdr port-forward service/vaultdr 8300:8200

Forwarding from 127.0.0.1:8300 -> 8200
Forwarding from [::1]:8300 -> 8200
Handling connection for 8300
Handling connection for 8300
Handling connection for 8300
Handling connection for 8300
Handling connection for 8300
Handling connection for 8300
